# Plan to Medal: Dogs vs. Cats Redux

Objective: Achieve ≤ 0.061 log-loss (bronze+) with a robust, efficient pipeline.

Workflow:
- Setup & sanity checks: verify data, counts, GPU, seeds, reproducibility.
- Baseline model fast: transfer learning with pretrained CNN (ResNet50/EfficientNet-B0), img_size≈224–256, mixed precision, light aug, single split to get quick LB.
- Cross-validation: 5-fold Stratified (by label), monitor log-loss, early stopping, save best per fold, average fold predictions.
- Inference: TTA (hflip, maybe 3-5 augmentations), predict probabilities for test, average over TTA and folds.
- Submission: create submission.csv with id, label.
- Iteration for medal: tune image size (320), stronger aug (RandomResizedCrop, color jitter), optimizer (AdamW), schedulers (OneCycle/Cosine), regularization (label smoothing), freeze/unfreeze strategy, higher-capacity models (tf_efficientnet_b3/b4), and increase epochs if needed.
- Efficiency: log progress, per-epoch timing, save checkpoints, early abort if overfitting, mixed precision, dataloader workers.

Milestones to request expert review:
1) After this plan.
2) After data loading + EDA.
3) After baseline training + first LB.
4) After CV results.
5) If training is slow/unstable, or score is poor.

Risks & mitigations:
- Overfitting: strong aug, label smoothing, early stopping.
- Time: start with quick baseline; scale up once LB is established.
- Package availability: prefer torchvision models; if available, consider timm for EfficientNet.

Next step:
- Implement environment checks and data listing; then request expert feedback on model choice and training recipe before heavy training.

In [1]:
# Environment checks, data listing, and basic setup
import os, sys, random, time, gc, math, glob, json, shutil, subprocess, importlib.util
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image

DATA_DIR = Path('.')
TRAIN_DIR = DATA_DIR / 'train'
TEST_DIR = DATA_DIR / 'test'
SAMPLE_SUB = DATA_DIR / 'sample_submission.csv'

print('Python', sys.version)
try:
    import torch
    print('PyTorch', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('GPU name:', torch.cuda.get_device_name(0))
        print('CUDA capability:', torch.cuda.get_device_capability(0))
except Exception as e:
    print('Torch not available:', e)

# Reproducibility
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass
set_seed(42)

# List files and counts
train_files = sorted(glob.glob(str(TRAIN_DIR / '*.jpg')))
test_files = sorted(glob.glob(str(TEST_DIR / '*.jpg')))
print(f'Train images: {len(train_files):,}')
print(f'Test images:  {len(test_files):,}')
print('Sample train:', train_files[:3])
print('Sample test :', test_files[:3])

# Build train DataFrame (label: dog=1, cat=0 inferred from filename prefix)
def infer_label_from_name(fp: str) -> int:
    name = os.path.basename(fp)
    if name.startswith('dog.'):
        return 1
    elif name.startswith('cat.'):
        return 0
    else:
        # Fallback if naming differs (shouldn't happen here)
        return -1

train_df = pd.DataFrame({'filepath': train_files})
train_df['label'] = train_df['filepath'].apply(infer_label_from_name)
assert (train_df['label'] >= 0).all(), 'Found unlabeled files in train directory.'
train_df['filename'] = train_df['filepath'].apply(os.path.basename)
print(train_df['label'].value_counts().rename({0:'cats',1:'dogs'}))

# Build test DataFrame with id extracted from filename (e.g., 1234.jpg -> 1234)
def extract_id(fp: str) -> int:
    name = os.path.basename(fp)
    stem = os.path.splitext(name)[0]
    return int(stem)

test_df = pd.DataFrame({'filepath': test_files})
test_df['id'] = test_df['filepath'].apply(extract_id)
test_df['filename'] = test_df['filepath'].apply(os.path.basename)
print('Test id range:', int(test_df['id'].min()), '->', int(test_df['id'].max()))

# Sanity: read a few random images to catch corruption
def is_image_ok(fp: str) -> bool:
    try:
        with Image.open(fp) as im:
            im.verify()  # quick check
        return True
    except Exception:
        return False

sample_check = random.sample(train_files, k=min(50, len(train_files)))
bad = [fp for fp in sample_check if not is_image_ok(fp)]
print(f'Quick corruption check on {len(sample_check)} train imgs -> bad: {len(bad)}')
if bad:
    print('Example bad files:', bad[:5])

# Check sample submission format
if SAMPLE_SUB.exists():
    ss = pd.read_csv(SAMPLE_SUB)
    print('Sample submission head:')
    print(ss.head())
    assert set(ss.columns) == {'id','label'}, 'sample_submission must have columns id,label'
else:
    print('sample_submission.csv not found')

# Check timm availability; we will install if missing in a later step
timm_available = importlib.util.find_spec('timm') is not None
print('timm available:', timm_available)

# Persist small metadata for next steps
META_PATH = Path('data_meta.json')
with open(META_PATH, 'w') as f:
    json.dump({
        'n_train': len(train_files),
        'n_test': len(test_files),
        'class_counts': train_df['label'].value_counts().to_dict(),
        'timm_available': timm_available
    }, f)
print('Wrote metadata to', META_PATH)

Python 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
Torch not available: No module named 'torch'
Train images: 22,500
Test images:  2,500
Sample train: ['train/cat.0.jpg', 'train/cat.1.jpg', 'train/cat.10.jpg']
Sample test : ['test/1.jpg', 'test/10.jpg', 'test/100.jpg']
label
dogs    11258
cats    11242
Name: count, dtype: int64
Test id range: 1 -> 2500
Quick corruption check on 50 train imgs -> bad: 0
Sample submission head:
   id  label
0   1    0.5
1   2    0.5
2   3    0.5
3   4    0.5
4   5    0.5
timm available: False
Wrote metadata to data_meta.json


In [2]:
# Install PyTorch (CUDA 12.1) and timm if missing
import importlib.util, sys, subprocess

def run(cmd):
    print('> ', ' '.join(cmd), flush=True)
    subprocess.check_call(cmd)

torch_spec = importlib.util.find_spec('torch')
timm_spec = importlib.util.find_spec('timm')

if torch_spec is None:
    run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
    run([sys.executable, '-m', 'pip', 'install', '--index-url', 'https://download.pytorch.org/whl/cu121', 'torch', 'torchvision', 'torchaudio'])
else:
    import torch
    print('Found torch', torch.__version__)

if timm_spec is None:
    run([sys.executable, '-m', 'pip', 'install', 'timm>=0.9.16'])
else:
    import timm
    print('Found timm', timm.__version__)

import torch, torchvision
print('Torch:', torch.__version__)
print('Torchvision:', torchvision.__version__)
try:
    import timm
    print('timm:', timm.__version__)
except Exception as e:
    print('timm import failed:', e)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))

>  /usr/bin/python3.11 -m pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00


>  /usr/bin/python3.11 -m pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio


Looking in indexes: https://download.pytorch.org/whl/cu121


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/780.5 MB ? eta -:--:--

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40/40 [timm]


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.8.0 which is incompatible.


Torch: 2.5.1+cu121
Torchvision: 0.20.1+cu121


/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


timm: 1.0.19
CUDA available: True
GPU: Tesla T4


In [ ]:
# Baseline training: 80/20 stratified split, EfficientNet-B0 @224, AMP, OneCycle, early stopping
import os, time, math, random, gc
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm

# Ensure HF cache is writable (avoid read-only /app/.cache)
HF_CACHE_DIR = str(Path('./hf_cache').absolute())
os.makedirs(HF_CACHE_DIR, exist_ok=True)
os.environ['HF_HOME'] = HF_CACHE_DIR
os.environ['HUGGINGFACE_HUB_CACHE'] = HF_CACHE_DIR

# Use objects created in cell 1 (train_df, test_df). If not present, rebuild quickly.
if 'train_df' not in globals():
    from pathlib import Path
    import glob, os
    TRAIN_DIR = Path('train'); TEST_DIR = Path('test')
    train_files = sorted(glob.glob(str(TRAIN_DIR / '*.jpg')))
    def infer_label_from_name(fp: str) -> int:
        name = os.path.basename(fp)
        return 1 if name.startswith('dog.') else 0
    train_df = pd.DataFrame({'filepath': train_files})
    train_df['label'] = train_df['filepath'].apply(infer_label_from_name)
    train_df['filename'] = train_df['filepath'].apply(os.path.basename)
    test_files = sorted(glob.glob(str(TEST_DIR / '*.jpg')))
    def extract_id(fp: str) -> int:
        name = os.path.basename(fp)
        return int(os.path.splitext(name)[0])
    test_df = pd.DataFrame({'filepath': test_files})
    test_df['id'] = test_df['filepath'].apply(extract_id)
    test_df['filename'] = test_df['filepath'].apply(os.path.basename)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
def set_seed(s=SEED):
    random.seed(s); np.random.seed(s); torch.manual_seed(s);
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(s);
        torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False
set_seed()

# Config
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 8
LR = 3e-4
WD = 1e-4
LABEL_SMOOTH = 0.05
PATIENCE = 2
NUM_WORKERS = 4

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_tfms = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0), interpolation=T.InterpolationMode.BICUBIC),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])
val_tfms = T.Compose([
    T.Resize(int(IMG_SIZE*1.14), interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

class ImageDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tfm, is_train=True):
        self.df = df.reset_index(drop=True)
        self.tfm = tfm
        self.is_train = is_train
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        fp = row['filepath']
        try:
            img = Image.open(fp).convert('RGB')
        except Exception:
            # In case of corrupt, return a black image of proper size
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
        img = self.tfm(img)
        if self.is_train:
            label = torch.tensor(row['label'], dtype=torch.float32)
            return img, label
        else:
            return img

# Stratified 80/20 split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, val_idx = next(sss.split(train_df['filename'], train_df['label']))
df_tr = train_df.iloc[train_idx].reset_index(drop=True)
df_val = train_df.iloc[val_idx].reset_index(drop=True)
print(f'Train size: {len(df_tr):,}, Val size: {len(df_val):,}')

ds_tr = ImageDataset(df_tr, train_tfms, is_train=True)
ds_val = ImageDataset(df_val, val_tfms, is_train=True)
dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
dl_val = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# Model (set cache_dir to writable path to avoid read-only filesystem issues)
model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True, num_classes=1, cache_dir=HF_CACHE_DIR)
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

# OneCycle over total steps
steps_per_epoch = len(dl_tr)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=steps_per_epoch,
    pct_start=0.15, anneal_strategy='cos', div_factor=25.0, final_div_factor=1e4
)

scaler = GradScaler(enabled=(device=='cuda'))

def sigmoid_np(x):
    return 1.0 / (1.0 + np.exp(-x))

def smooth_targets(y, eps=LABEL_SMOOTH):
    if eps <= 0:
        return y
    return y * (1.0 - eps) + 0.5 * eps

def evaluate(model, dl):
    model.eval()
    all_logits = []
    all_targets = []
    with torch.no_grad():
        for xb, yb in dl:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            with autocast(enabled=(device=='cuda')):
                logits = model(xb).squeeze(1)
            all_logits.append(logits.detach().float().cpu().numpy())
            all_targets.append(yb.detach().float().cpu().numpy())
    logits = np.concatenate(all_logits)
    targets = np.concatenate(all_targets)
    probs = sigmoid_np(logits)
    probs = np.clip(probs, 1e-6, 1-1e-6)
    return log_loss(targets, probs), probs

best_loss = float('inf')
epochs_no_improve = 0
start_time = time.time()
for epoch in range(1, EPOCHS+1):
    model.train()
    t0 = time.time()
    running_loss = 0.0
    for i, (xb, yb) in enumerate(dl_tr):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        yb_sm = smooth_targets(yb, LABEL_SMOOTH)
        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device=='cuda')):
            logits = model(xb).squeeze(1)
            loss = criterion(logits, yb_sm)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        running_loss += loss.item()
        if (i+1) % 50 == 0 or (i+1)==len(dl_tr):
            print(f'Epoch {epoch}/{EPOCHS} | Step {i+1}/{len(dl_tr)} | Loss {running_loss/(i+1):.4f} | LR {scheduler.get_last_lr()[0]:.6f}', flush=True)
    val_loss, _ = evaluate(model, dl_val)
    print(f'Epoch {epoch} done in {time.time()-t0:.1f}s | Val logloss: {val_loss:.5f}', flush=True)
    # Early stopping
    if val_loss < best_loss - 1e-5:
        best_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'baseline_b0_best.pth')
        print(f'  Improved best -> {best_loss:.5f} (model saved)', flush=True)
    else:
        epochs_no_improve += 1
        print(f'  No improve counter: {epochs_no_improve}/{PATIENCE}', flush=True)
        if epochs_no_improve >= PATIENCE:
            print('Early stopping triggered.', flush=True)
            break
print(f'Training completed in {int(time.time()-start_time)}s. Best val logloss: {best_loss:.5f}')

# Load best and predict validation and test with hflip TTA
state_dict = torch.load('baseline_b0_best.pth', map_location=device)
model.load_state_dict(state_dict)
model.eval()

def predict_dl(dl, tta_hflip=True):
    preds = []
    with torch.no_grad():
        for xb in dl:
            if isinstance(xb, (tuple, list)):
                xb = xb[0]
            xb = xb.to(device, non_blocking=True)
            with autocast(enabled=(device=='cuda')):
                logit1 = model(xb).squeeze(1)
                if tta_hflip:
                    xb_flip = torch.flip(xb, dims=[3])
                    logit2 = model(xb_flip).squeeze(1)
                    logit = (logit1 + logit2) / 2.0
                else:
                    logit = logit1
            preds.append(logit.float().cpu().numpy())
    logits = np.concatenate(preds)
    probs = sigmoid_np(logits)
    probs = np.clip(probs, 1e-6, 1-1e-6)
    return probs

# Validation predictions (for sanity check)
dl_val_eval = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
val_probs = predict_dl(dl_val_eval, tta_hflip=True)
val_loss_tta = log_loss(df_val['label'].values, val_probs)
print(f'Validation logloss with TTA: {val_loss_tta:.5f}')

# Test predictions
ds_test = ImageDataset(test_df, val_tfms, is_train=False)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_probs = predict_dl(dl_test, tta_hflip=True)

sub = pd.DataFrame({'id': test_df['id'].values, 'label': test_probs})
sub = sub.sort_values('id').reset_index(drop=True)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with shape', sub.shape, 'and head:')
print(sub.head())

# Cleanup
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

Train size: 18,000, Val size: 4,500


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(
/tmp/ipykernel_61/585201399.py:128: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device=='cuda'))


/tmp/ipykernel_61/585201399.py:168: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device=='cuda')):


/app/.pip-target/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
